This notebook is written by taking in reference to  this notebookk(https://www.kaggle.com/code/xinruizhuang/skin-lesion-classification-acc-90-pytorch/notebook#Step-2.-Model-building)


# Importing Libraries

In [5]:
import pandas as pd
import os
from glob import glob
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image
from torchvision import models,transforms
from torch.utils.data import DataLoader,Dataset
import torch
from torch import optim,nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split



# Data Processing

In [ ]:
df = pd.read_csv("final_df.csv")
dff = df.sample(1000)

In [6]:
dff

,Unnamed: 0,lesion_id,image_id,dx,dx_type,age,sex,localization,path,full_name,cell_type_idx
2920,2920,HAM_0007497,ISIC_0026829,bcc,histo,60.0,female,back,images\HAM10000_images_part_1\ISIC_0026829.jpg,Basal cell carcinoma,1
166,166,HAM_0001385,ISIC_0033770,bkl,histo,50.0,female,neck,images\HAM10000_images_part_2\ISIC_0033770.jpg,Benign keratosis-like lesions,2
2821,2821,HAM_0004495,ISIC_0024743,bcc,histo,65.0,female,lower extremity,images\HAM10000_images_part_1\ISIC_0024743.jpg,Basal cell carcinoma,1
7073,7073,HAM_0007138,ISIC_0031501,nv,histo,25.0,male,hand,images\HAM10000_images_part_2\ISIC_0031501.jpg,Melanocytic nevi,4
8730,8730,HAM_0006514,ISIC_0026559,nv,histo,60.0,female,lower extremity,images\HAM10000_images_part_1\ISIC_0026559.jpg,Melanocytic nevi,4
...,...,...,...,...,...,...,...,...,...,...,...
5910,5910,HAM_0000011,ISIC_0024678,nv,follow_up,45.0,female,trunk,images\HAM10000_images_part_1\ISIC_0024678.jpg,Melanocytic nevi,4
9053,9053,HAM_0000836,ISIC_0030061,nv,histo,45.0,female,back,images\HAM10000_images_part_2\ISIC_0030061.jpg,Melanocytic nevi,4
2750,2750,HAM_0005933,ISIC_0024634,bcc,histo,80.0,female,back,images\HAM10000_images_part_1\ISIC_0024634.jpg,Basal cell carcinoma,1
7168,7168,HAM_0000304,ISIC_0034083,nv,histo,45.0,female,abdomen,images\HAM10000_images_part_2\ISIC_0034083.jpg,Melanocytic nevi,4


In [19]:
dff['age'].fillna(int(df['age'].mean()),inplace=True)

In [20]:
data_dir= "output_images"
combined_images_path = glob(os.path.join(data_dir, '*', '*.jpg'))

combined_images_path_dict = {os.path.splitext(
            os.path.basename(x)
        )[0]: x 
        for x in combined_images_path
        }

lesion_full_name_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

dff['path'] = dff['image_id'].map(combined_images_path_dict.get)
dff['full_name'] = dff['dx'].map(lesion_full_name_dict.get)

dff['cell_type_idx'] = pd.Categorical(dff['full_name']).codes

In [21]:
dff["path"].isna().sum()

930

Resizing the image to get the standarized input

In [23]:
imgs = []
labels = []  # Assuming you have a corresponding list of labels
for i in tqdm(range(len(combined_images_path))):
    img = cv2.imread(combined_images_path[i])
    img = cv2.resize(img, (64, 64))  # Resize to 64x64
    imgs.append(img)
    labels.append([i])  # Add the corresponding label

# Convert to NumPy arrays
imgs_array = np.array(imgs)
labels_array = np.array(labels)

# Save images and labels
np.save("images_64x64.npy", imgs_array)
np.save("labels.npy", labels_array)

print("Preprocessed data saved successfully!")

100%|██████████| 700/700 [00:04<00:00, 140.36it/s]

Preprocessed data saved successfully!


In [24]:
loaded_imgs = np.load('images_32X32.npy')

In [25]:
stacked_imgs = np.stack(loaded_imgs, axis=3)

In [ ]:
def compute_img_mean_std(images):
    means, stdevs = [], []
    imgs = images.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

mean, std = compute_img_mean_std(images=stacked_imgs)

train_transform = transforms.Compose([transforms.Resize((64,64)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                        transforms.ToTensor(), transforms.Normalize(mean, std)])
# define the transformation of the val images.
val_transform = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])



normMean = [0.7631431, 0.5457683, 0.5701589]
normStd = [0.14080769, 0.15254283, 0.16990419]


In [ ]:
class Skin_Cancer_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [44]:
df_undupp = df.groupby('lesion_id').count()
# now we filter out lesion_id's that have only one image associated with it
df_undup = df_undupp[df_undupp['image_id'] == 1]
df_undup.reset_index(inplace=True)

# here we identify lesion_id's that have duplicate images and those that have only one image.
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# create a new colum that is a copy of the lesion_id column
df['duplicates'] = df['lesion_id']
# apply the function to this new column
df['duplicates'] = df['duplicates'].apply(get_duplicates)

df_undup = df[df['duplicates'] == 'unduplicated']
df_undup.shape
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape

(1103, 12)

In [45]:
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# identify train and val rows
# create a new colum that is a copy of the image_id column
df['train_or_val'] = df['image_id']
# apply the function to this new column
df['train_or_val'] = df['train_or_val'].apply(get_val_rows)
# filter out train rows
df_train = df[df['train_or_val'] == 'train']
print(len(df_train))
print(len(df_val))

8912
1103


In [46]:
df_train.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,full_name,cell_type_idx,duplicates,train_or_val
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0027419.jpg,Benign keratosis-like lesions,2,duplicated,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0025030.jpg,Benign keratosis-like lesions,2,duplicated,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0026769.jpg,Benign keratosis-like lesions,2,duplicated,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0025661.jpg,Benign keratosis-like lesions,2,duplicated,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,images\HAM10000_images_part_2\ISIC_0031633.jpg,Benign keratosis-like lesions,2,duplicated,train


In [47]:
df_train.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,full_name,cell_type_idx,duplicates,train_or_val
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0027419.jpg,Benign keratosis-like lesions,2,duplicated,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0025030.jpg,Benign keratosis-like lesions,2,duplicated,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0026769.jpg,Benign keratosis-like lesions,2,duplicated,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,images\HAM10000_images_part_1\ISIC_0025661.jpg,Benign keratosis-like lesions,2,duplicated,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,images\HAM10000_images_part_2\ISIC_0031633.jpg,Benign keratosis-like lesions,2,duplicated,train


there are 2 options of augmentation we can do but we will go with first one

In [23]:
# Copy fewer class to balance the number of 7 classes
data_aug_rate = [15,10,5,50,0,40,5]
for i in range(7):
    if data_aug_rate[i]:
        df_train=df_train.append([df_train.loc[df_train['cell_type_idx'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
df_train['full_name'].value_counts()

Melanocytic nevi                  5822
dermatofibroma                    1067
Benign keratosis-like lesions     1011
Dermatofibroma                     963
Basal cell carcinoma               958
Vascular lesions                   903
Actinic keratoses                  891
Name: full_name, dtype: int64


C:\Users\abuza\AppData\Local\Temp\ipykernel_11916\1981679223.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append([df_train.loc[df_train['full_name'] == class_name]] * (aug_rate - 1), ignore_index=True)


In [49]:
# Set target count for each class
target_count = 1000

# New DataFrame to hold the balanced data
balanced_df = pd.DataFrame()

# Process each class
for class_name, count in df_train["full_name"].value_counts().items():
    if count > target_count:
        # Downsample to 1,000 samples
        sampled_data = df_train[df_train["full_name"] == class_name].sample(target_count, random_state=42)
    else:
        # Upsample to 1,000 samples
        sampled_data = df_train[df_train["full_name"] == class_name].sample(target_count, replace=True, random_state=42)
    
    # Append to the balanced DataFrame
    balanced_df = pd.concat([balanced_df, sampled_data])

# Check the new class distribution
print(balanced_df["full_name"].value_counts())


Melanocytic nevi                  1000
dermatofibroma                    1000
Benign keratosis-like lesions     1000
Basal cell carcinoma              1000
Actinic keratoses                 1000
Vascular lesions                  1000
Dermatofibroma                    1000
Name: full_name, dtype: int64


Converting our data to dataloader for convience of processing

In [50]:
training_set = Skin_Cancer_Dataset(balanced_df, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=16, shuffle=True, num_workers=4)
# Same for the validation set:
validation_set = Skin_Cancer_Dataset(df_val.head(100), transform=train_transform)
val_loader = DataLoader(validation_set, batch_size=16, shuffle=False, num_workers=4)

In [28]:
balanced_df.to_csv("balanced_df.csv", index=False)
df_val.head(100).to_csv("df_val.csv", index=False)

# Save the DataLoader state
torch.save(train_loader, "train_loader_v1.pth")
torch.save(val_loader, "val_loader_v1.pth")

# Model Defition

In [52]:
class SimpleResNet32(nn.Module):
    def __init__(self, num_classes):
        super(SimpleResNet32, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(32, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        for _ in range(blocks):
            layers.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ))
            in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
resnet_model = SimpleResNet32(num_classes=7)

# Model Training

In [27]:
def train_model(model, dataloader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [53]:
def save_checkpoint(model, optimizer, epoch, running_loss, save_dir):
  """
  Saves a checkpoint of the model and optimizer state.

  Args:
      model: The model to save.
      optimizer: The optimizer to save.
      epoch: The current epoch number.
      running_loss: The current running loss.
      save_dir: The directory to save the checkpoint.
  """
  checkpoint = {
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'running_loss': running_loss,
  }
  torch.save(checkpoint, os.path.join(save_dir, f'checkpoint_{epoch}.pt'))

from tqdm import tqdm

def train_model(model, dataloader, criterion, optimizer, num_epochs, device, save_dir=None):
  """
  Trains a model with tqdm progress bar and checkpointing.

  Args:
      model: The model to be trained.
      dataloader: The data loader for training data.
      criterion: The loss function.
      optimizer: The optimizer for updating model weights.
      num_epochs: The number of epochs for training.
      device: The device to train on (CPU or GPU).
      save_dir (optional): The directory to save checkpoints (default: None).
  """
  model.to(device)
  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    pbar = tqdm(total=len(dataloader))  # Initialize tqdm progress bar

    for i, (inputs, labels) in enumerate(dataloader):
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)
      print(f"-----------{running_loss}")


      # Update progress bar and potentially save checkpoint
      pbar.update(1)
      if (i + 1) % 1000 == 0 and save_dir is not None:
        save_checkpoint(model, optimizer, epoch + 1, running_loss, save_dir)

    epoch_loss = running_loss / len(dataloader.dataset)
    pbar.close()  # Close progress bar after epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.001)
train_model(resnet_model, train_loader, criterion, optimizer, num_epochs=10, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


  0%|          | 0/438 [00:00<?, ?it/s]

In [ ]:
import mlflow.pytorch

# Replace 'c509cc57b036461fa0b48...' with your specific run ID
logged_model_path = "mlruns\\mlruns\\0\\c509cc57b036461fa0b485f2b9f4b700\\artifacts\\best_model"

# Load the model
model = mlflow.pytorch.load_model(logged_model_path)

# Use the model for inference
output = model(validation_set)


In [15]:
import os

mlruns_path = "mlruns\\mlruns\\0"
for run_id in os.listdir(mlruns_path):
    run_path = os.path.join(mlruns_path, run_id)
    if os.path.isdir(run_path):
        metrics_path = os.path.join(run_path, "metrics")
        params_path = os.path.join(run_path, "params")

        # Read metrics
        for metric_file in os.listdir(metrics_path):
            with open(os.path.join(metrics_path, metric_file), 'r') as f:
                print(f"{metric_file}: {f.read()}")

        # Read parameters
        for param_file in os.listdir(params_path):
            with open(os.path.join(params_path, param_file), 'r') as f:
                print(f"{param_file}: {f.read()}")


train_acc: 1736878811604 0.5823143115942029 1
1736879024059 0.7407029991948471 2
1736879230169 0.7871125201288245 3
1736879437513 0.8238476247987118 4
1736879636706 0.8288295088566827 5
1736879839620 0.8582804951690822 6
1736880041552 0.8641304347826086 7
1736880242496 0.8631491545893719 8
1736880443673 0.8818438003220612 9
1736880645928 0.9065393518518519 10

train_loss: 1736878811603 1.130077346607491 1
1736879024057 0.6938984562401418 2
1736879230167 0.5584175835053126 3
1736879437511 0.4646117427834758 4
1736879636702 0.4407988178923174 5
1736879839618 0.3745421838981134 6
1736880041550 0.36428303443999205 7
1736880242494 0.3771179804095515 8
1736880443672 0.31786069956918556 9
1736880645927 0.25115534333788136 10

val_acc: 1736878811606 0.43968413978494625 1
1736879024060 0.5967741935483871 2
1736879230170 0.6596102150537634 3
1736879437517 0.5658602150537634 4
1736879636708 0.6643145161290323 5
1736879839622 0.6280241935483871 6
1736880041554 0.5962701612903226 7
1736880242497 0.